In computer programming, the interpreter pattern is a design pattern that specifies <b> how to evaluate sentences in a language </b>. The basic idea is to have a class for each symbol (terminal or nonterminal) in a specialized computer language. The syntax tree of a sentence in the language is an instance of the composite pattern and is used to evaluate (interpret) the sentence for a client

### source - https://en.wikipedia.org/wiki/Interpreter_pattern

* <b>Uses</b>
 - Specialized database query languages such as SQL.
 - Specialized computer languages that are often used to describe communication protocols.
 - Most general-purpose computer languages actually incorporate several specialized languages.

## Class Diagram
![Class Diagram](http://www.goprod.bouhours.net/images/patterns/en/41992565150af956e41e90.svg)

### Design components

* **AbstractExpression (Expression)**: Declares an interpret() operation that all nodes (terminal and nonterminal) in the AST overrides.
* **TerminalExpression (NumberExpression)**: Implements the interpret() operation for terminal expressions.
* **NonterminalExpression (AdditionExpression, SubtractionExpression, and MultiplicationExpression)**: Implements the interpret() operation for all nonterminal expressions.
* **Context (String)**: Contains information that is global to the interpreter. It is this String expression with the Postfix notation that has to be interpreted and parsed.
* **Client (ExpressionParser)**: Builds (or is provided) the AST assembled from TerminalExpression and NonTerminalExpression. The Client invokes the interpret() operation.

### Example 

Context = “Select ```COLUMN``` from ```TABLE``` where ```CONDITION```”

In [2]:
## Abstract Expression
from abc import ABCMeta, abstractstaticmethod


class Expression(metaclass=ABCMeta):
    @abstractstaticmethod
    def interpret(self, ctx):
            pass
        
class Where(Expression):
    __filter__ = None

    def __init__(self, _filter):
        self.__filter__ = _filter

    def interpret(self, ctx):
        ctx.set_filter(self.__filter__)
        return ctx.search()
    

class From(Expression):
    __table__ = None
    __where__ = None
    
    def __init__(self, table, where:Where=None):
        self.__table__ = table
        self.__where__ = where

    def interpret(self, ctx): 
        ctx.set_table(self.__table__)
        if (self.__where__ == None):
            return ctx.search()
        
        return self.__where__.interpret(ctx)
    
        
class Select(Expression):
    __column__ = None
    __from__  = None

    def __init__(self, column, _from: From):
        self.__column__ = column
        self.__from__ = _from

    def interpret(self, ctx):
        ctx.set_column(self.__column__)
        return self.__from__.interpret(ctx)


In [3]:
class Context():
    tables = dict()
    data = list()
    data.append(("John", "Doe"))
    data.append(("Jan", "Kowalski"))
    data.append(("Dominic", "Doom"))

    tables["people"] = data

    colIndex = -1;
    whereFilter = None
    matchAnyString = lambda s : len(s[0]) > 0
    
    def set_filter(self, _filter):
        self.whereFilter = _filter
    
    def set_column(self, column):
        self.columns = self.column_mapper(column)
    
    def set_table(self,table):
        self.table = table
    
    def column_mapper(self, column):
        column_map = ["name","surname", "*"]
        return column_map.index(column) 

    def clear(self):
        column = ""
        self.whereFilter = self.matchAnyString

    def search(self):
        if(self.whereFilter == None ):
            return [row[self.columns] for row in self.tables[self.table]]
        
        if self.columns == 2:
            return [row for row in self.tables[self.table]]
            
        result = [row[self.columns] for row in self.tables[self.table] if self.whereFilter(row[self.columns])]
        
        self.clear()

        return result

In [5]:
query = Select("name", From("people"))
ctx = Context()
result = query.interpret(ctx)
print(result)

query3 = Select("name", From("people", Where(lambda name : name.lower().startswith("j"))))
result3 = query3.interpret(ctx)
print(result3)

query2 = Select("*", From("people"))
result2 = query2.interpret(ctx)
print(result2)

['John', 'Jan', 'Dominic']
['John', 'Jan']
[('John', 'Doe'), ('Jan', 'Kowalski'), ('Dominic', 'Doom')]
